# Lightning Channel via Batch Transaction

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND GRPC

In [ ]:
import sys

In [ ]:
# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from lnd_rpc import LndRpc, lnmsg, walletmsg, signrpc, signmsg

In [ ]:
ali = LndRpc(host='ali-lnd:10009',
             cert_path='/ali-lnd/tls.cert',
             macaroon_path='/tmp/ali.macaroon')

In [ ]:
bob = LndRpc(host='bob-lnd:10009',
             cert_path='/bob-lnd/tls.cert',
             macaroon_path='/tmp/bob.macaroon')

In [ ]:
aliNodePubKey = ali.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
aliNodePubKey

In [ ]:
bobNodePubKey = bob.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
bobNodePubKey

## Fund Ali and Bob

In [ ]:
import os
import bitcoin
from rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
brpc.getblockcount()

In [ ]:
aliAddr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
aliAddr

In [ ]:
bobAddr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
bobAddr

In [ ]:
# fund ali
_ = list(brpc.generatetoaddress(1, aliAddr.address))

In [ ]:
# fund bob
_ = list(brpc.generatetoaddress(1, bobAddr.address))

In [ ]:
brpc.createwallet('miner')
minerAddr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

In [ ]:
# unlock mined coins
_ = list(brpc.generatetoaddress(110, minerAddr))

In [ ]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

In [ ]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

## Create bid and ask

In [ ]:
import base64
import bitcoin.core as bc
import bitcoin.core.script as bs
from bitcoin.rpc import hexlify, unhexlify

### Ali taker

In [ ]:
# taker needs to pay premium to open channel
premiumAmount = int(0.001 * bc.COIN) # premium Ali is willing to pay
fundAmount = int(0.16 * bc.COIN) # requested inbound capacity

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): premiumAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)

In [ ]:
psbt = ali.wallet.FundPsbt(request=psbtRequest)
psbt

In [ ]:
psbtBase64 = base64.b64encode(psbt.funded_psbt).decode()

In [ ]:
brpc._proxy._call('analyzepsbt', psbtBase64)

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
psbtDecoded

In [ ]:
aliInputs = []
for vin in psbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    aliInputs.append(txin)
aliInputs

In [ ]:
# assume there are only 2 outputs
assert len(psbtDecoded['tx']['vout']) == 2

In [ ]:
aliChangeOutput = psbtDecoded['tx']['vout'][psbt.change_output_index]
aliChangeOutput = bc.CTxOut(int(aliChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(aliChangeOutput['scriptPubKey']['hex'])))

aliChangeOutput

In [ ]:
# Generate pubkey for channel funding
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq) # need to call twice?
aliKeyDesc

In [ ]:
takeRequest = {
    'nodeAddr': 'ali-lnd',
    'nodePubKey': aliNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'inputs': aliInputs,
    'change': aliChangeOutput,
    'multisigKey': aliKeyDesc # only need to send bytes of pub key and not key locator
}
takeRequest

### Bob maker

In [ ]:
premiumAmount = int(0.001 * bc.COIN) # premium Bob is willing to accept
fundAmount = int(0.16 * bc.COIN) # funding amount Bob is willing to provide

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): fundAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)
psbtRequest

In [ ]:
psbt = bob.wallet.FundPsbt(request=psbtRequest)
psbt

In [ ]:
psbtBase64 = base64.b64encode(psbt.funded_psbt).decode()

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
psbtDecoded['tx']

In [ ]:
bobInputs = []
for vin in psbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    bobInputs.append(txin)
bobInputs

In [ ]:
# assume there are only 2 outputs
assert len(psbtDecoded['tx']['vout']) == 2

In [ ]:
bobChangeOutput = psbtDecoded['tx']['vout'][psbt.change_output_index]
bobChangeOutput = bc.CTxOut(int(bobChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(bobChangeOutput['scriptPubKey']['hex'])))

bobChangeOutput

In [ ]:
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq) # need to call twice?
bobKeyDesc

In [ ]:
makeRequest = {
    'nodeAddr': 'bob-lnd',
    'nodePubKey': bobNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'inputs': bobInputs,
    'change': bobChangeOutput,
    'multisigKey': bobKeyDesc # only need to send bytes of pub key and not key locator
}
makeRequest

## Auctioneer

In [ ]:
import hashlib

### create batch funding transaction

In [ ]:
takerPubKey = takeRequest['multisigKey'].raw_key_bytes
makerPubKey = makeRequest['multisigKey'].raw_key_bytes

In [ ]:
# https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md#funding-transaction-output
# https://github.com/bitcoin/bips/blob/master/bip-0069.mediawiki
# if makerPubKey > takerPubKey:
if list(makerPubKey) > list(takerPubKey):
    pk1, pk2 = takerPubKey, makerPubKey
else:
    pk1, pk2 = makerPubKey, takerPubKey

In [ ]:
msigScript = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

In [ ]:
# convert to P2WSH
scriptPubKey = bc.CScript([bs.OP_0, hashlib.sha256(msigScript).digest()])
scriptPubKey

In [ ]:
assert scriptPubKey.is_witness_v0_scripthash()

In [ ]:
fundingOutput = bc.CTxOut(takeRequest['premiumAmount'] + makeRequest['fundAmount'],
                          scriptPubKey)
fundingOutput

In [ ]:
batchInputs = takeRequest['inputs'] + makeRequest['inputs']
batchOutputs = [fundingOutput, takeRequest['change'], makeRequest['change']]
fundingOutputIdx = 0

In [ ]:
batchTx = bc.CTransaction(batchInputs, batchOutputs)
batchTx

In [ ]:
batchTxId = batchTx.GetTxid()
batchTxId

### Response to matched taker and maker

In [ ]:
# something that is unique to the corresponding matched taker and maker
channelId = hashlib.sha256((bobNodePubKey + aliNodePubKey).encode()).hexdigest()

In [ ]:
takeResponse = {
    'batchTx': batchTx,
    'fundingOutputIdx': fundingOutputIdx,
    'channelId': channelId,
    'maker': {
        'nodeAddr': makeRequest['nodeAddr'],
        'nodePubKey': makeRequest['nodePubKey'],
        'premiumAmount': makeRequest['premiumAmount'],
        'fundAmount': makeRequest['fundAmount'],
        'remoteKey': makeRequest['multisigKey'].raw_key_bytes
    }
}

In [ ]:
makeResponse = {
    'batchTx': batchTx,
    'fundingOutputIdx': fundingOutputIdx,
    'channelId': channelId,
    'taker': {
        'nodeAddr': takeRequest['nodeAddr'],
        'nodePubKey': takeRequest['nodePubKey'],
        'premiumAmount': takeRequest['premiumAmount'],
        'fundAmount': takeRequest['fundAmount'],
        'remoteKey': takeRequest['multisigKey'].raw_key_bytes
    }
}

## Ali taker

In [ ]:
takeResponse

### Register funding shim to get ready for funding
this will also prompt taker to connect to maker

In [ ]:
maker = takeResponse['maker']
batchTx = takeResponse['batchTx']

In [ ]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=batchTx.GetTxid(),
                                output_index=takeResponse['fundingOutputIdx'])

In [ ]:
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=aliKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=aliKeyDesc.key_loc.key_family,
        key_index=aliKeyDesc.key_loc.key_index))

In [ ]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=maker['fundAmount'] + maker['premiumAmount'],
    chan_point=chan_point,
    local_key=local_key,
    remote_key=maker['remoteKey'],
    pending_chan_id=unhexlify(takeResponse['channelId']),
    thaw_height=0 # set 0 for simplicity
)

In [ ]:
shim = lnmsg.FundingShim(chan_point_shim=chan_point_shim)

In [ ]:
ftm = lnmsg.FundingTransitionMsg(shim_register=shim)

In [ ]:
# empty response
ali.lnd.FundingStateStep(ftm)

### Connect to maker

In [ ]:
addr = lnmsg.LightningAddress(
    pubkey=maker['nodePubKey'],
    host=maker['nodeAddr']
)

In [ ]:
connect_peer_req = lnmsg.ConnectPeerRequest(
    addr=addr
)

In [ ]:
ali.lnd.ConnectPeer(connect_peer_req)

## Bob maker

In [ ]:
makeResponse

### Initiate open channel request

In [ ]:
taker = makeResponse['taker']
batchTx = makeResponse['batchTx']

In [ ]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=batchTx.GetTxid(),
                                output_index=takeResponse['fundingOutputIdx'])

In [ ]:
chan_point = lnmsg.ChannelPoint(
    funding_txid_bytes=batchTx.GetTxid(),
    output_index=makeResponse['fundingOutputIdx']
)

In [ ]:
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=bobKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=bobKeyDesc.key_loc.key_family,
        key_index=bobKeyDesc.key_loc.key_index
    )
)

In [ ]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=taker['fundAmount'] + taker['premiumAmount'],
    chan_point=chan_point,
    local_key=local_key,
    remote_key=taker['remoteKey'],
    pending_chan_id=unhexlify(makeResponse['channelId']),
    thaw_height=0 # for simplicity
)

In [ ]:
funding_shim = lnmsg.FundingShim(chan_point_shim=chan_point_shim)

In [ ]:
open_chan_req = lnmsg.OpenChannelRequest(
    node_pubkey=unhexlify(taker['nodePubKey']),
    local_funding_amount=taker['fundAmount'] + taker['premiumAmount'],
    push_sat=taker['premiumAmount'], # premium is pushed to the remote end
    funding_shim=funding_shim
)

In [ ]:
chanEventStream = bob.lnd.OpenChannel(open_chan_req)

In [ ]:
next(chanEventStream)

In [ ]:
# at this point Ali and Bob should have a pending channel
bob.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

## Sign batch transaction

### Ali taker

In [ ]:
takeResponse['batchTx']

In [ ]:
pbst = brpc._proxy._call('converttopsbt', hexlify(takeResponse['batchTx'].serialize()))
pbst

In [ ]:
psbtRequest = walletmsg.SignPsbtRequest(psbt=base64.b64decode(pbst))

aliSignedPsbt = ali.wallet.SignPsbt(psbtRequest)
aliSignedPsbt

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', base64.b64encode(aliSignedPsbt.signed_psbt).decode())
psbtDecoded

In [ ]:
takerUpdate = {
    'signed_psbt': base64.b64encode(aliSignedPsbt.signed_psbt).decode()
}

### Bob maker

In [ ]:
makeResponse['batchTx']

In [ ]:
pbst = brpc._proxy._call('converttopsbt', hexlify(makeResponse['batchTx'].serialize()))
pbst

In [ ]:
psbtRequest = walletmsg.SignPsbtRequest(psbt=base64.b64decode(pbst))

bobSignedPsbt = bob.wallet.SignPsbt(psbtRequest)
bobSignedPsbt

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', base64.b64encode(bobSignedPsbt.signed_psbt).decode())
psbtDecoded

In [ ]:
makerUpdate = {
    'signed_psbt': base64.b64encode(bobSignedPsbt.signed_psbt).decode()
}

### Finalize and publish batch transaction

### combine

In [ ]:
combinedPsbt = brpc._proxy._call('combinepsbt', [takerUpdate['signed_psbt'], makerUpdate['signed_psbt']])
combinedPsbt

In [ ]:
brpc._proxy._call('decodepsbt', combinedPsbt)

### finalize

In [ ]:
res = brpc._proxy._call('finalizepsbt', combinedPsbt)
res

### submit

In [ ]:
tx = bc.CTransaction.deserialize(unhexlify(res['hex']))
tx

In [ ]:
batchTxId2 = brpc.sendrawtransaction(tx)
batchTxId2

In [ ]:
assert batchTxId == batchTxId2

In [ ]:
_ = list(brpc.generatetoaddress(6, minerAddr))

In [ ]:
brpc.getrawtransaction(batchTxId2, verbose=True)

## Check channel

In [ ]:
# should match the channel point
bc.b2lx(batchTxId2)

In [ ]:
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
ali.lnd.ListChannels(lnmsg.ListChannelsRequest())